In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import nltk

# Download NLTK resources (if not downloaded already)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/tmp/ipykernel_8989/587196773.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-29 20:27:06.266703: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 20:27:07.046219: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2

True

In [2]:
df=pd.read_csv("pcl.csv")

In [3]:
df

,Project Description,Cover Letter
0,I need a Python developer for a long-term part...,I am writing to express my interest in the Pyt...
1,Need to write a file handling program in Pytho...,I hope this email finds you well. I am writing...
2,I need someone to help me with Agent to contro...,I am writing to express my keen interest in th...
3,We are seeking a skilled Python coder to assis...,I am writing to express my interest in the Pyt...
4,"I have a scenario with a set of questions, you...",I am writing to express my interest in the sce...
5,i have a python script. It is download a file ...,I am writing to express my interest in the opp...
6,I need an individual to scrape data from ali e...,I am reaching out to express my interest in th...
7,\n\nJoin our dynamic team as a Junior Python D...,I am excited to apply for the Junior Python De...
8,\n\nAre you passionate about Python programmin...,\n\nI am writing to express my interest in the...


In [4]:
# Text preprocessing function
def preprocess_text(text):
    # Remove newline characters
    text = text.replace('\n', ' ')

    # Tokenization
    tokens = word_tokenize(text)
    # Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    stripped = [word.translate(table) for word in tokens]
    # Remove non-alphabetic tokens
    words = [word for word in stripped if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)


In [5]:



# Extract project descriptions and cover letters
project_descriptions = df['Project Description'].tolist()
cover_letters = df['Cover Letter'].tolist()

# Preprocess the project descriptions and cover letters
preprocessed_project_descriptions = [preprocess_text(desc) for desc in project_descriptions]
preprocessed_cover_letters = [preprocess_text(letter) for letter in cover_letters]

In [6]:


# Tokenize input and output sequences
tokenizer_desc = Tokenizer()
tokenizer_desc.fit_on_texts(preprocessed_project_descriptions)
tokenizer_cover = Tokenizer()
tokenizer_cover.fit_on_texts(preprocessed_cover_letters)

# Convert text sequences to integer sequences
X = tokenizer_desc.texts_to_sequences(preprocessed_project_descriptions)
y = tokenizer_cover.texts_to_sequences(preprocessed_cover_letters)

# Pad sequences to ensure uniform length
max_seq_length = max(max(len(seq) for seq in X), max(len(seq) for seq in y))
X = pad_sequences(X, maxlen=max_seq_length, padding='post')
y = pad_sequences(y, maxlen=max_seq_length, padding='post')

In [7]:

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Model architecture
def create_model(input_vocab_size, output_vocab_size, max_seq_length, hidden_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_vocab_size, hidden_units),
        tf.keras.layers.LSTM(hidden_units),
        tf.keras.layers.RepeatVector(max_seq_length),
        tf.keras.layers.LSTM(hidden_units, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(output_vocab_size, activation='softmax'))
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    return model

In [9]:




# Create model
input_vocab_size = len(tokenizer_desc.word_index) + 1
output_vocab_size = len(tokenizer_cover.word_index) + 1
hidden_units = 256
model = create_model(input_vocab_size, output_vocab_size, max_seq_length, hidden_units)

# Train model
batch_size = 64
epochs = 5
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)


Epoch 1/5


2024-04-11 21:50:56.451655: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 5.9493 - val_loss: 5.8519
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - loss: 5.8715 - val_loss: 5.6485
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 5.7085 - val_loss: 5.0070
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - loss: 5.1938 - val_loss: 4.0683
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - loss: 4.4623 - val_loss: 3.8853


In [10]:

# Save the trained model
model.save('cover_letter_generator_model.h5')


In [11]:

# Load the trained model
model = tf.keras.models.load_model('cover_letter_generator_model.h5')


In [14]:
# New project description
new_project_description = ["I need a python developer"]

# Preprocess the new project description
preprocessed_new_project_description = preprocess_text(new_project_description[0])

# Convert the preprocessed project description to sequences using the tokenizer
new_project_sequence = tokenizer_desc.texts_to_sequences([preprocessed_new_project_description])

# Pad the sequence to match the input sequence length used during training
new_project_padded_sequence = pad_sequences(new_project_sequence, maxlen=max_seq_length, padding='post')


# Predict the cover letter for the new project description
predicted_cover_letter_sequence = model.predict(new_project_padded_sequence)

# Convert the predicted sequence back to text using the tokenizer for cover letters
predicted_cover_letter_text = []
for sequence in predicted_cover_letter_sequence[0]:  # Take the first sequence (as there's only 1)
    # Sample a token based on its probability distribution
    sampled_token_index = np.random.choice(len(sequence), p=sequence)
    # Convert the index to its corresponding word
    word = tokenizer_cover.index_word.get(sampled_token_index, '')
    # Append the word to the cover letter text
    predicted_cover_letter_text.append(word)

# Join the words to form the predicted cover letter text
predicted_cover_letter_text = ' '.join(predicted_cover_letter_text)

# Print the generated cover letter
print("Generated Cover Letter:")
print(predicted_cover_letter_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Generated Cover Letter:
warm prospect fulfill opportunity capable      appropriately    efficiency  network expertise    express  journey         various strengthen response price perfectly    compression  scheduling open    innovation confident   critically  look completion sale   extract enthusiastic   desire   development     team address handling completion strong  moreover   my contributing  contact contact file duration detail reviewed desire  assignment coding scenariobased price agent    help   provided    worked  system  support   clear highquality  maintains   python      code communication contribute  detail  genuinely  some performance   file   crosschecking  align genuinely  committed   gz genuine       proficiency service    background developer    storage 


In [ ]:
!python3 --version